In [ ]:
pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download 'omkarmanohardalvi/lungs-disease-dataset-4-types'

Dataset URL: https://www.kaggle.com/datasets/omkarmanohardalvi/lungs-disease-dataset-4-types
License(s): unknown
100% 2.02G/2.02G [01:06<00:00, 38.6MB/s]
100% 2.02G/2.02G [01:06<00:00, 32.7MB/s]


In [ ]:
!unzip /content/lungs-disease-dataset-4-types.zip

Streaming output truncated to the last 5000 lines.
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0673-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0675-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0678-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0680-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0682-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0683-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0684-0001-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0686-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0690-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0692-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0693-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0695-0001.jpeg  
  inflating: Lung Disease Dataset/train/Norm

In [ ]:
!unzip /content/lungs-disease-dataset-4-types.zip

Streaming output truncated to the last 5000 lines.
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0673-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0675-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0678-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0680-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0682-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0683-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0684-0001-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0686-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0690-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0692-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0693-0001.jpeg  
  inflating: Lung Disease Dataset/train/Normal/NORMAL2-IM-0695-0001.jpeg  
  inflating: Lung Disease Dataset/train/Norm

In [ ]:
!pip install -q flwr[simulation] torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
from sklearn.metrics import precision_score, recall_score
import time
from typing import List
from collections import OrderedDict
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import time
import polyline 
from polyline import decode

import flwr as fl

from flwr.common import Metrics

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

NUM_CLIENTS = 10

#IID Dataset partition

def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
    [
        transforms.Resize((512, 512)),  # Resize all images to 512x512
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))
    ]
    )
    trainset = ImageFolder(root="/content/Lung Disease Dataset/train", transform=transform)
    testset = ImageFolder(root="/content/Lung Disease Dataset/test", transform=transform)
    val= ImageFolder(root="/content/Lung Disease Dataset/val", transform=transform)
    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    remaining_samples = len(trainset) - partition_size * num_clients

# Adjust the partition sizes to accommodate the remaining samples
    lengths = [partition_size] * num_clients
    for i in range(remaining_samples):
        lengths[i] += 1

# Create the datasets with random splits
    datasets = random_split(trainset, lengths, generator=torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        # len_val = len(ds) // 10  # 10 % validation set
        # len_train = len(ds) - len_val
        # lengths = [len_train, len_val]
        # ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(trainset, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        # Adjusted input size based on the output size after convolutional layers
        self.fc1 = nn.Linear(16 * 125 * 125, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        #print("After conv1 and maxpool:", x.size())
        x = self.pool(F.relu(self.conv2(x)))
        #print("After conv2 and maxpool:", x.size())
        # Adjusted based on the output size from conv2
        x = x.view(-1, 16 * 125 * 125)
        #print("After view:", x.size())
        x = F.relu(self.fc1(x))
        #print("After fc1:", x.size())
        x = F.relu(self.fc2(x))
        #print("After fc2:", x.size())
        x = self.fc3(x)
        #print("After fc3:", x.size())
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss = criterion(net(images), labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

def encode_parameters(params):
    flat_params = np.concatenate([p.flatten() for p in params])
    int_params = (flat_params * 1e6).astype(int)
    encoded = polyline.encode([(v, 0) for v in int_params])

    # Log the encoded data for debugging
    print("Encoded Parameters:", encoded)
    return encoded
    
def decode_parameters(encoded, shapes):
    decoded = polyline.decode(encoded)
    deltas = np.array([v[0] for v in decoded])
    int_params = np.cumsum(deltas) / 1e6  # Reverse scaling
    split_indices = np.cumsum([np.prod(shape) for shape in shapes])[:-1]
    param_tensors = [torch.tensor(arr.reshape(shape)) for arr, shape in zip(np.split(int_params, split_indices), shapes)]
    return param_tensors

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self,config):
      params = [p.detach().cpu().numpy() for p in self.model.parameters()]
      encoded = encode_parameters(params)
      return [encoded]  # Wrap it in a list


    def set_parameters(self, parameters):
      encoded = parameters[0]  # Extract first element if passed as list
      shapes = [p.shape for p in self.model.parameters()]
      decoded_params = decode_parameters(encoded, shapes)
      for param, new_param in zip(self.model.parameters(), decoded_params):
          param.data = new_param.float()

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        num_labels = 0
        for _, labels in self.trainloader:
            num_labels += len(labels)
        data_label_ratio = num_labels / len(self.trainloader.dataset)

        # Add timestamp and data_label_ratio to metrics
        metrics = {"timestamp": time.time(), "data_label_ratio": data_label_ratio}

        return get_parameters(self.net), len(self.trainloader.dataset), metrics

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
params = get_parameters(Net())

def test_with_metrics(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    loss_fn = nn.CrossEntropyLoss()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        total_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        precision = precision_score(all_labels, all_predictions, average='weighted')
        recall = recall_score(all_labels, all_predictions, average='weighted')

    return total_loss, accuracy, precision, recall

def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    start_time = time.time()  # Record start time
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy, precision, recall = test_with_metrics(net, valloader)
    end_time = time.time()  # Record end time
    round_time = end_time - start_time  # Calculate round time
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy} / precision {precision} / recall {recall}")
    print(f"Time taken for round {server_round}: {round_time} seconds")
    return loss, {"accuracy": accuracy, "precision": precision, "recall": recall}


class FceFed(FedAvg):
    def __init__(
        self,
        fraction_fit: float = 0.3,
        fraction_evaluate: float = 0.3, #Sample 50% of available clients for evaluation
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 1, #Allow evaluation with as few as 1 client
        min_available_clients: int = 2,

    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate
        self.initial_lr = 0.001
        self.mu = 0.1
        self.decay_factor = 0.01
    def __repr__(self) -> str:
        return "FceFed"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)



    def aggregate_fit(self, server_round, results, failures):
        current_timestamp = time.time()
        weights_results = []
        for client, fit_res in results:
            parameters = parameters_to_ndarrays(fit_res.parameters)
            client_timestamp = fit_res.metrics["timestamp"]
            staleness = current_timestamp - client_timestamp

            eta_k = self.initial_lr * np.exp(-self.decay_factor * staleness)
            fairness_factor = max(0, 1 - self.mu * staleness)
            data_ratio = fit_res.metrics["data_label_ratio"]
            client_weight = eta_k * data_ratio * fairness_factor


        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=200), 
    strategy=FceFed(),
    client_resources=client_resources,
)



Training on cpu using PyTorch 2.2.1+cu121 and Flower 1.8.0


INFO :      Starting Flower simulation, config: num_rounds=200, no round_timeout
INFO:flwr:Starting Flower simulation, config: num_rounds=200, no round_timeout
/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-05-11 15:52:18,738	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 7933729998.0, 'object_store_memory': 3966864998.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 7933729998.0, 'object_store_memory': 3966864998.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Fl

Server-side evaluation loss 144.67549514770508 / accuracy 0.19940476190476192 / precision 0.039762259070294785 / recall 0.19940476190476192
Time taken for round 0: 201.59303545951843 seconds
(ClientAppActor pid=3788) [Client 3] fit, config: {}
(ClientAppActor pid=3788) Epoch 1: train loss 0.025416869670152664, accuracy 0.6665014866204163
(ClientAppActor pid=3787) [Client 6] fit, config: {}
(ClientAppActor pid=3788) [Client 7] fit, config: {}
(ClientAppActor pid=3787) Epoch 1: train loss 0.02659541741013527, accuracy 0.6546085232903865
(ClientAppActor pid=3788) Epoch 1: train loss 0.0268133245408535, accuracy 0.652791542781632


INFO :      aggregate_fit: received 3 results and 0 failures
INFO:flwr:aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (1, 64.01619146764278, {'accuracy': 0.6502976190476191, 'precision': 0.685893773430389, 'recall': 0.6502976190476191}, 4314.021474862)
INFO:flwr:fit progress: (1, 64.01619146764278, {'accuracy': 0.6502976190476191, 'precision': 0.685893773430389, 'recall': 0.6502976190476191}, 4314.021474862)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)
INFO:flwr:configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 64.01619146764278 / accuracy 0.6502976190476191 / precision 0.685893773430389 / recall 0.6502976190476191
Time taken for round 1: 197.80743885040283 seconds
(ClientAppActor pid=3788) [Client 3] evaluate, config: {}
(ClientAppActor pid=3788) [Client 2] evaluate, config: {} [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
